In [2]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats 

In [63]:
def load_data():
    std_dev = 1
    mean = 5
    x1 = np.linspace(1, 9, 100)
    x2 = np.linspace(3, 11, 100)
    y = ((x1 > 6) & (x2 > 9)).astype(int)
    X = np.stack((x1, x2), axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=10)
    y_train = np.reshape(y_train, (y_train.shape[0], 1))
    y_test = np.reshape(y_test, (y_test.shape[0], 1))
    return X_train, X_test, y_train, y_test

In [97]:
def initialize_params(X_train):
    n = X_train.shape[1]
    w = np.random.randn(n, 1) * 0.1
    b = np.zeros((1, 1))
    params = {'w':w, 'b':b}
    return params

In [98]:
def sigmoid(z):
    a = 1/(1 + np.exp(-z))
    return a

In [120]:
def forward_prop(X, params):
    w = params['w']
    b = params['b']
    z = X @ w + b
    inter_vals = {'z':z}
    yhat = sigmoid(z)
    return yhat, inter_vals

In [121]:
def calculate_cost(yhat, y):
    m = y.shape[0]
    losses = y * np.log(yhat) + (1 - y) * np.log(1 - yhat)
    print(f'losses {losses}')
    cost = -np.sum(losses, axis=0, keepdims=True)/m
    return cost

In [122]:
def back_prop(y, yhat, inter_vals):
    m = y.shape[0]
    z = inter_vals['z']
    dc_dyhat = (-1/m) * ((y/yhat) - ((1-y)/(1-yhat)))
    dyhat_dz = sigmoid(z) * (1 - sigmoid(z))
    dc_dz = dc_dyhat * dyhat_dz
    dc_db = np.sum(dc_dz, axis=0, keepdims=True)
    

In [100]:
X_train, X_test, y_train, y_test = load_data()

In [101]:
params = initialize_params(X_train)

In [102]:
yhat, inter_vals = forward_prop(X_train, params)

In [123]:
X = np.array([[0.8, 0.9], [0.3, 0.5]])
w = np.array([[0.5], [0.75]])
b = np.array([[1]])
params = {'w':w,'b':b}
yhat, inter_vals = forward_prop(X, params)

In [124]:
yhat

array([[0.88844946],
       [0.82127358]])

In [125]:
y = np.array([[1], [0]])

In [118]:
cost = calculate_cost(yhat, y)

losses [[-0.11827752]
 [-1.721899  ]]


In [119]:
cost

array([[0.92008826]])

In [ ]:
grads = back_prop(yhat, y, inter_vals)